In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')

In [4]:
#read data with the new columns
#current status -> sorted by studId and time, added current_level column
df = pd.read_csv("Data/FullData_AddedColumns.csv")

#RUN THIS ALL THE TIME
df['userId'] = df['userId'].astype('str')
df['object_id'] = df['object_id'].astype('str')
df['timestamp'] = df['timestamp'].astype('str')

In [46]:
df_temp = pd.read_csv("data/StudentLevelFeatures.csv")
df_temp = df_temp.groupby(['current_level'], as_index=False).aggregate(['count'])['userId'] > 50
df_temp.to_csv("Data/temp_level.csv")

#<---READ LEVELS TO BE PROCESSED-->

#df_temp = pd.read_csv('Data/LevelFeatures.csv')
#levels = df_temp['current_level']
print "Processing", len(levels), "levels"

In [5]:
#ATTEMPT features - at the finest level of granularity
for level in levels:
    level_name = level.replace('levels\\/', '')
    level_name= level_name.replace('\\/', '_')
    level_name = level_name.replace('.level', '')
    level_name = level_name.replace(' ', '_')
    print "Processing", level_name
    
    df_stud = df[df["current_level"] == level]
    print "Number of log instances", len(df_stud)
    print "unique students", len(df_stud["userId"].unique())
    print "Total number of attempts by all students", len(df_stud[(df_stud["event"] == 'Level Start')])
    print "Total successful attempts", len(df_stud[(df_stud["badge"] != '.') & (df_stud["badge"] != 'none')])
    print "Total quit attempts", len(df_stud[df_stud["badge"] == 'none'])
    print "Class distribution of quit", float(len(df_stud[df_stud["badge"] == 'none']))/float(len(df_stud[(df_stud["event"] == 'Level Start')]))
    
    #Remove OUTLIERS
    #Remove student with less than 10 log entries -> VERIFY
    print "Remove student with less than 10 log entries..."
    df_temp = df_stud["userId"].groupby(df_stud["userId"]).value_counts()
    df_temp = df_temp[df_temp > 10]
    user_list = [df_temp.index[i][0] for i in range(len(df_temp))]
    print "Removed", len(df_stud["userId"].unique())-len(df_temp)
    df_stud = df_stud[df_stud["userId"].isin(user_list)]
    print "New number of log instances", len(df_stud)

    #Remove "Menu Focus" events - refers transition between levels
    print "Remove menu focus event entries (refers transition between levels).."
    df_stud = df_stud[df_stud["event"] != 'Menu Focus']
    print "New number of log instances", len(df_stud)

    #???Remove "Summary Data" events - no corresponding student actions
    print "Remove summary data event entries (no corresponding student actions).."
    df_stud = df_stud[df_stud["event"] != 'Summary Data']
    print "New number of log instances", len(df_stud)
    
    #remove records between level start and level end events - irrelevant for the analysis
    stud = None
    for idx, row in df_stud.iterrows():
        #new student
        if row['userId'] != stud:
            if row['event'] != 'Level Start':
                df_stud.drop(idx, inplace=True) #deletes events before level start
                continue
            stud = row['userId']
            start = True #level started
            continue

        if row['event'] == 'Level Start':
            start = True

        if start == False:
            df_stud.drop(idx, inplace=True) #deletes events after level end and before next level start

        if row['event'] == 'Level End':
            start = False
    
    #primary columns from data
    print "Remove records between level start and level end events - irrelevant for the analysis.."
    print "New number of log instances", len(df_stud)
                 
    df_time_series = df_stud[["userId", "utc_time", "timestamp", "badge", "event", "object_id", "primary_id", 
                          "elapsed", 'strength', 'mass', 'position', 'width', 'height', 'length']]
    print len(df_time_series)
                 
    #ADD LABELS
    #1a. quit label - 1 if quit
    #IMPORTANT alternative - 1 if quit in next 60 seconds -- predict probability that the student quits the attempt he/she is pursuing in the next 60 seconds

    for idx, row in df_time_series.iterrows():
        #new attempt
        if row['event'] == 'Level Start':
            idx_list = [] #empty list

        idx_list.append(idx)    
        if row['event'] == 'Level End':
            if row['badge'] == 'none': #indicated quit
                df_time_series.set_value(idx_list, 'quit', 1)
            else:
                df_time_series.set_value(idx_list, 'quit', 0)
    
    print "number of quit log instances",len(df_time_series[df_time_series["quit"] == 1])
    print "class distribution",float(len(df_time_series[df_time_series["quit"] == 1]))/float(len(df_time_series))
    
    #2. time in the attempt
    stud = None
    for idx, row in df_time_series.iterrows():
        #new student
        if row['userId'] != stud:
            if row['event'] != 'Level Start':
                continue
            stud = row['userId']   

        if row['event'] == 'Level Start':
            time_step = 0
        else:
            time_step += int(row["timestamp"]) - last_timestamp   

        last_timestamp = int(row["timestamp"])

        df_time_series.set_value(idx,'time_step', time_step)

        if row['event'] == 'Level End':
            time_step = 0
    
    #1b. quit label - 1 if quit in next 2 minutes -- predict probability that the student quits the attempt he/she is pursuing in the next 60 seconds
    for idx, row in df_time_series.iterrows():
        #new attempt
        if row['event'] == 'Level Start':
            time_list = {} #empty list

        time_list[idx] = row['time_step']
        if row['event'] == 'Level End':
            if row['badge'] == 'none': #indicated quit
                quit = [k for k in time_list if time_list[k] >= (row['time_step'] - (1000*120))]
                df_time_series.set_value(quit, 'quit_x_sec', 1)
                no_quit = [k for k in time_list if time_list[k] < row['time_step'] - (1000*120)]            
                df_time_series.set_value(no_quit, 'quit_x_sec', 0)
            else:
                df_time_series.set_value(time_list.keys(), 'quit_x_sec', 0)
      
    #features on level events - info on student interaction with the level
    #3. level restart by student overall, per attempt
    #4. ball reset overall, per attempt
    #5. badges overall
    #6. number of quits before
    #7. attempt ID
    #8. number of level pauses
    #9. last pause time
    #10. total pause time
    #11. paused - yes or no

    stud = None
    for idx, row in df_time_series.iterrows():
        #new student
        if row['userId'] != stud:
            stud = row['userId']  
            level_restarts_overall = 0
            ball_resets_overall = 0
            level_restarts_attempt = 0
            ball_resets_attempt = 0
            badges = set()
            quits_num = 0
            attempt = 0
            number_pauses_overall = 0
            number_pauses_attempt = 0
            last_pause_duration = 0
            total_pause_duration_overall = 0
            total_pause_duration_attempt = 0
            last_pause_time_step = 0
            last_level_restart_time_step = 0
            last_ball_reset_time_step = 0
            #TO-DO: Add badge features for silver vs gold and agents used  

        #new attempt
        if row['event'] == 'Level Start':
            attempt += 1
            level_restarts_attempt = 0
            ball_resets_attempt = 0
            last_pause_duration = 0
            total_pause_duration_attempt = 0
            number_pauses_attempt = 0
            last_pause_time_step = 0
            last_level_restart_time_step = 0
            last_ball_reset_time_step = 0

        if row['event'] == "Lost" and row['object_id'] == '1':
            ball_resets_attempt += 1
            ball_resets_overall += 1
            last_ball_reset_time_step = row['time_step']
        if row['event'] == "Level Restart":
            level_restarts_attempt += 1  
            level_restarts_overall += 1
            last_level_restart_time_step = row['time_step']
        if row['badge'] != '.' and row['badge'] != 'none':
            badges.add(row['badge'])
        if row['event'] == 'Level End' and row['badge'] == 'none':
            quits_num += 1
        if row['event'] == 'Level Pause':
            df_time_series.set_value(idx,'paused', 1)
            number_pauses_overall += 1
            number_pauses_attempt += 1
            last_pause_duration = int(row['elapsed'])
            total_pause_duration_overall += int(row['elapsed'])
            total_pause_duration_attempt += int(row['elapsed'])
            last_pause_time_step = row['time_step']
        else:
            df_time_series.set_value(idx,'paused', 0)

        df_time_series.set_value(idx,'level_restarts_overall', level_restarts_overall)
        df_time_series.set_value(idx,'ball_resets_overall', ball_resets_overall)
        df_time_series.set_value(idx,'level_restarts_attempt', level_restarts_attempt)
        df_time_series.set_value(idx,'ball_resets_attempt', ball_resets_attempt)
        df_time_series.set_value(idx,'badges_overall', len(badges))
        df_time_series.set_value(idx,'quits_num', quits_num)
        df_time_series.set_value(idx,'attempt', attempt)
        df_time_series.set_value(idx,'number_pauses_overall', number_pauses_overall)
        df_time_series.set_value(idx,'number_pauses_attempt', number_pauses_attempt)
        df_time_series.set_value(idx,'last_pause_duration', last_pause_duration)
        df_time_series.set_value(idx,'total_pause_duration_overall', total_pause_duration_overall)
        df_time_series.set_value(idx,'total_pause_duration_attempt', total_pause_duration_attempt)
        df_time_series.set_value(idx,'time_elapsed_last_pause', row['time_step'] - last_pause_time_step)
        df_time_series.set_value(idx,'time_elapsed_last_level_restart', row['time_step'] - last_level_restart_time_step)
        df_time_series.set_value(idx,'time_elapsed_last_ball_reset', row['time_step'] - last_ball_reset_time_step)

    #features on play events  - gives precise info on the player's approach to the solution 
    #Agent created - check word doc and code all of it
    stud = None
    for idx, row in df_time_series.iterrows():
        #new student
        if row['userId'] != stud:
            stud = row['userId']  
            total_freeform_overall = 0
            total_freeform_attempt = 0
            total_pin_overall = 0
            total_pin_attempt = 0
            total_nudge_overall = 0
            total_nudge_attempt = 0
            total_pendulum_overall = 0
            total_pendulum_attempt = 0
            total_lever_overall = 0
            total_lever_attempt = 0
            total_ramp_overall = 0
            total_ramp_attempt = 0
            total_springboard_overall = 0
            total_springboard_attempt = 0
            total_pulldown_overall = 0 
            total_pulldown_attempt = 0
            total_stacking_overall = 0 
            total_stacking_attempt = 0
            total_diver_overall = 0 
            total_diver_attempt = 0
            freeform_objs = set()
            pendulum_objs = set()
            lever_objs = set()
            ramp_objs = set()
            springboard_objs = set()
            pin_objs = set()

        #new attempt
        if row['event'] == 'Level Start':
            total_freeform_attempt = 0
            total_pin_attempt = 0
            total_nudge_attempt = 0
            total_pendulum_attempt = 0
            total_lever_attempt = 0
            total_ramp_attempt = 0
            total_springboard_attempt = 0
            total_pulldown_attempt = 0
            total_stacking_attempt = 0
            total_diver_attempt = 0
            freeform_objs = set()
            pendulum_objs = set()
            lever_objs = set()
            ramp_objs = set()
            springboard_objs = set()
            pin_objs = set()

        if row["event"] == "Erase":
            df_time_series.set_value(idx,'erase_now', 1)
            obj = row["object_id"]
            if obj in freeform_objs:
                freeform_objs.remove(obj)
            if obj in pendulum_objs:
                pendulum_objs.remove(obj)
            if obj in lever_objs:
                lever_objs.remove(obj)
            if obj in ramp_objs:
                ramp_objs.remove(obj)
            if obj in springboard_objs:
                springboard_objs.remove(obj)
            if obj in pin_objs:
                pin_objs.remove(obj)
        else:
            df_time_series.set_value(idx,'erase_now', 0)

        if row['event'] == "Draw Freeform":
            df_time_series.set_value(idx,'draw_freeform_now', 1)
            total_freeform_overall += 1 #including the ones erased
            total_freeform_attempt += 1#including the ones ereased
            object_mass = row['mass'] 
            object_position = row['position']
            object_width = row["width"]
            object_height = row["height"]
            object_length = row["length"]
            freeform_objs.add(row["object_id"])
        else:
            df_time_series.set_value(idx,'draw_freeform_now', 0)

        if row['event'] == "Draw Pin":
            df_time_series.set_value(idx,'draw_pin_now', 1)
            total_pin_overall += 1 #including the ones erased
            total_pin_attempt += 1#including the ones ereased
            object_position = row['position']
            pin_objs.add(row["object_id"])
        else:
            df_time_series.set_value(idx,'draw_pin_now', 0)

        if row['event'] == "Nudge":
            df_time_series.set_value(idx,'nudge_now', 1)
            total_nudge_overall += 1 #including the ones erased
            total_nudge_attempt += 1#including the ones ereased
        else:
            df_time_series.set_value(idx, 'nudge_now', 0)

        if row['event'] == "Pendulum Object":
            df_time_series.set_value(idx,'draw_pendulum_now', 1)
            total_pendulum_overall += 1 #including the ones erased
            total_pendulum_attempt += 1#including the ones ereased
            pendulum_objs.add(row["primary_id"]) #same as freeform object_id - becomes pendulum when displays a behavior
            if row['strength'] == 'strong':
                object_strength = 1
            if row['strength'] == 'weak':
                object_strength = 0
        else:
            df_time_series.set_value(idx,'draw_pendulum_now', 0)

        if row['event'] == "Pendulum Strike":
            df_time_series.set_value(idx,'pendulum_strike_now', 1)
        else:
            df_time_series.set_value(idx,'pendulum_strike_now', 0)

        if row['event'] == "Lever":
            df_time_series.set_value(idx,'draw_lever_now', 1)
            total_lever_overall += 1 #including the ones erased
            total_lever_attempt += 1#including the ones ereased
            lever_objs.add(row["primary_id"]) #same as freeform object_id - becomes pendulum when displays a behavior
            if row['strength'] == 'strong':
                object_strength = 1
            if row['strength'] == 'weak':
                object_strength = 0
        else:
            df_time_series.set_value(idx,'draw_lever_now', 0)

        if row['event'] == "Springboard":
            df_time_series.set_value(idx,'draw_springboard_now', 1)
            total_springboard_overall += 1 #including the ones erased
            total_springboard_attempt += 1#including the ones ereased
            springboard_objs.add(row["primary_id"]) #same as freeform object_id - becomes pendulum when displays a behavior
            if row['strength'] == 'strong':
                object_strength = 1
            if row['strength'] == 'weak':
                object_strength = 0
        else:
            df_time_series.set_value(idx,'draw_springboard_now', 0)

        if row['event'] == "Ramp":
            df_time_series.set_value(idx,'draw_ramp_now', 1)
            total_ramp_overall += 1 #including the ones erased
            total_ramp_attempt += 1#including the ones ereased
            ramp_objs.add(row["primary_id"]) #same as freeform object_id - becomes pendulum when displays a behavior
            if row['strength'] == 'strong':
                object_strength = 1
            if row['strength'] == 'weak':
                object_strength = 0
        else:
            df_time_series.set_value(idx,'draw_ramp_now', 0)

        if row['event'] == "Pulldown":
            total_pulldown_overall += 1 
            total_pulldown_attempt += 1
            df_time_series.set_value(idx,'pulldown_now', 1)
        else:
            df_time_series.set_value(idx,'pulldown_now', 0)

        if row['event'] == "Stacking":
            total_stacking_overall += 1 
            total_stacking_attempt += 1
            df_time_series.set_value(idx,'stacking_now', 1)
        else:
            df_time_series.set_value(idx,'stacking_now', 0)

        if row['event'] == "Diver":
            total_diver_overall += 1 
            total_diver_attempt += 1
            df_time_series.set_value(idx,'diver_now', 1)
        else:
            df_time_series.set_value(idx,'diver_now', 0)

        if row['event'] == "Draw Rope":
            df_time_series.set_value(idx, 'draw_rope_now', 1)
        else:
            df_time_series.set_value(idx,'draw_rope_now', 0)

        if row['event'] == "Lost":
            df_time_series.set_value(idx, 'lost_now', 1)
        else:
            df_time_series.set_value(idx,'lost_now', 0)

        if row['event'] == "Click":
            df_time_series.set_value(idx, 'click_now', 1)
        else:
            df_time_series.set_value(idx,'click_now', 0)

        if row['event'] == "Collision":
            df_time_series.set_value(idx, 'collision_now', 1)
        else:
            df_time_series.set_value(idx,'collision_now', 0)


        df_time_series.set_value(idx,'current_num_freefrom_objs', len(freeform_objs))
        df_time_series.set_value(idx,'current_num_freefrom_objs', len(pin_objs))
        df_time_series.set_value(idx,'current_num_pendulum_objs', len(pendulum_objs))
        df_time_series.set_value(idx,'current_num_lever_objs', len(lever_objs))
        df_time_series.set_value(idx,'current_num_ramp_objs', len(ramp_objs))
        df_time_series.set_value(idx,'current_num_springboard_objs', len(springboard_objs))
        df_time_series.set_value(idx,'total_freeform_overall', total_freeform_overall)
        df_time_series.set_value(idx,'total_freeform_attempt', total_freeform_attempt)
        df_time_series.set_value(idx,'total_pin_overall', total_pin_overall)
        df_time_series.set_value(idx,'total_pin_attempt', total_pin_attempt)
        df_time_series.set_value(idx,'total_nudge_overall', total_nudge_overall)
        df_time_series.set_value(idx,'total_nudge_attempt', total_nudge_attempt)
        df_time_series.set_value(idx,'total_pendulum_overall', total_pendulum_overall)
        df_time_series.set_value(idx,'total_pendulum_attempt', total_pendulum_attempt)
        df_time_series.set_value(idx,'total_lever_overall', total_lever_overall)
        df_time_series.set_value(idx,'total_lever_attempt', total_lever_attempt)
        df_time_series.set_value(idx,'total_ramp_overall', total_ramp_overall)
        df_time_series.set_value(idx,'total_ramp_attempt', total_ramp_attempt)
        df_time_series.set_value(idx,'total_springboard_overall', total_springboard_overall)
        df_time_series.set_value(idx,'total_springboard_attempt', total_springboard_attempt)
        df_time_series.set_value(idx,'total_pulldown_overall', total_pulldown_overall)
        df_time_series.set_value(idx,'total_pulldown_attempt', total_pulldown_attempt)
        df_time_series.set_value(idx,'total_stacking_overall', total_stacking_overall)
        df_time_series.set_value(idx,'total_stacking_attempt', total_stacking_attempt)
        df_time_series.set_value(idx,'total_diver_overall', total_diver_overall)
        df_time_series.set_value(idx,'total_diver_attempt', total_diver_attempt)

        #object specific values will be recorded only for the log instance when it was first drawn
        object_mass = 0
        object_strength = 0
        object_position = 0
        object_width = 0
        object_height = 0
        object_length = 0
    
    filename = "Data/LevelFeatures/FullNewLevels/"+level_name+".csv"
    df_time_series.to_csv(filename, header=True, index=False)
    print "---------------------------------------------------"
    print "---------------------------------------------------"

Processing p2_downhill
Number of log instances 15381
unique students 125
Total number of attempts by all students 183
Total successful attempts 166
Total quit attempts 17
Class distribution of quit 0.0928961748634
Remove student with less than 10 log entries...
Removed 0
New number of log instances 15381
Remove menu focus event entries (refers transition between levels)..
New number of log instances 15100
Remove summary data event entries (no corresponding student actions)..
New number of log instances 9310
Remove records between level start and level end events - irrelevant for the analysis..
New number of log instances 9302
9302
number of quit log instances 680
class distribution 0.0731025585896
---------------------------------------------------
---------------------------------------------------
Processing p2_lead_the_ball
Number of log instances 10858
unique students 124
Total number of attempts by all students 177
Total successful attempts 171
Total quit attempts 6
Class distribu

In [6]:
def update_df_agg(df_agg, df_temp, count):
    df_agg.set_value(count, 'userId', df_temp.iloc[-1]['userId']) #not included
    df_agg.set_value(count, 'total_records', len(df_temp)) #not included
    df_agg.set_value(count, 'quit', df_temp.iloc[-1]['quit'])
    df_agg.set_value(count, 'record_num', record_num)
    df_agg.set_value(count, 'num_pauses', np.sum(df_temp['paused']))
    df_agg.set_value(count, 'level_restarts_overall', np.max(df_temp['level_restarts_overall']))
    df_agg.set_value(count, 'ball_resets_overall',np.max(df_temp['ball_resets_overall']))
    df_agg.set_value(count, 'level_restarts_attempt', np.max(df_temp['level_restarts_attempt']))
    df_agg.set_value(count, 'ball_resets_attempt',np.max(df_temp['ball_resets_attempt']))
    df_agg.set_value(count, 'badges_overall', np.max(df_temp['badges_overall']))
    df_agg.set_value(count, 'quits_num',np.max(df_temp['quits_num']))
    df_agg.set_value(count, 'attempt',np.max(df_temp['attempt']))
    df_agg.set_value(count, 'number_pauses_overall', np.max(df_temp['number_pauses_overall']))
    df_agg.set_value(count, 'number_pauses_attempt', np.max(df_temp['number_pauses_attempt']))
    df_agg.set_value(count, 'last_pause_duration',np.max(df_temp['last_pause_duration']))
    df_agg.set_value(count, 'total_pause_duration_overall', np.max(df_temp['total_pause_duration_overall']))
    df_agg.set_value(count, 'total_pause_duration_attempt', np.max(df_temp['total_pause_duration_attempt']))
    df_agg.set_value(count, 'time_elapsed_last_pause', df_temp.iloc[-1]['time_elapsed_last_pause'])
    df_agg.set_value(count, 'time_elapsed_last_level_restart',df_temp.iloc[-1]['number_pauses_overall'])
    df_agg.set_value(count, 'time_elapsed_last_ball_reset', df_temp.iloc[-1]['number_pauses_overall'])
    df_agg.set_value(count, 'num_draw_freeform', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_draw_pin', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_nudge', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_draw_pendulum', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_draw_lever', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_draw_springboard', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'num_draw_ramp', np.sum(df_temp['draw_freeform_now']))
    df_agg.set_value(count, 'current_num_freefrom_objs', df_temp.iloc[-1]['current_num_freefrom_objs'])
    df_agg.set_value(count, 'current_num_pendulum_objs', df_temp.iloc[-1]['current_num_pendulum_objs'])
    df_agg.set_value(count, 'current_num_lever_objs', df_temp.iloc[-1]['current_num_lever_objs'])
    df_agg.set_value(count, 'current_num_ramp_objs', df_temp.iloc[-1]['current_num_ramp_objs'])
    df_agg.set_value(count, 'current_num_springboard_objs', df_temp.iloc[-1]['current_num_springboard_objs'])
    df_agg.set_value(count, 'total_freeform_overall', df_temp.iloc[-1]['total_freeform_overall'])
    df_agg.set_value(count, 'total_freeform_attempt', df_temp.iloc[-1]['total_freeform_attempt'])
    df_agg.set_value(count, 'total_pin_overall',df_temp.iloc[-1]['total_pin_overall'])
    df_agg.set_value(count, 'total_pin_attempt',df_temp.iloc[-1]['total_pin_attempt'])
    df_agg.set_value(count, 'total_nudge_overall',df_temp.iloc[-1]['total_nudge_overall'])
    df_agg.set_value(count, 'total_nudge_attempt',df_temp.iloc[-1]['total_nudge_attempt'])
    df_agg.set_value(count, 'total_pendulum_overall',df_temp.iloc[-1]['total_pendulum_overall'])
    df_agg.set_value(count, 'total_pendulum_attempt',df_temp.iloc[-1]['total_pendulum_attempt'])
    df_agg.set_value(count, 'total_lever_overall',df_temp.iloc[-1]['total_lever_overall'])
    df_agg.set_value(count, 'total_lever_attempt',df_temp.iloc[-1]['total_lever_attempt'])
    df_agg.set_value(count, 'total_ramp_overall',df_temp.iloc[-1]['total_freeform_overall'])
    df_agg.set_value(count, 'total_ramp_attempt',df_temp.iloc[-1]['total_ramp_attempt'])
    df_agg.set_value(count, 'total_springboard_overall',df_temp.iloc[-1]['total_springboard_overall'])
    df_agg.set_value(count, 'total_springboard_attempt',df_temp.iloc[-1]['total_springboard_attempt'])
    df_agg.set_value(count, 'total_pulldown_overall',df_temp.iloc[-1]['total_pulldown_overall'])
    df_agg.set_value(count, 'total_pulldown_attempt',df_temp.iloc[-1]['total_pulldown_attempt'])
    df_agg.set_value(count, 'total_stacking_overall',df_temp.iloc[-1]['total_stacking_overall'])
    df_agg.set_value(count, 'total_stacking_attempt',df_temp.iloc[-1]['total_stacking_attempt'])
    df_agg.set_value(count, 'total_diver_overall',df_temp.iloc[-1]['total_diver_overall'])
    df_agg.set_value(count, 'total_diver_attempt',df_temp.iloc[-1]['total_diver_attempt'])

    return df_agg

In [13]:
#FEATURE AGGREGATION
import pandas as pd
import numpy as np

TIME_BIN = 60
MAX_TIME = 1200
df_level_full = pd.read_csv("Data/StudentLevelFeatures.csv")
df_level_full['userId'] = df_level_full['userId'].astype('str')

for level in levels:
    df_time_series = pd.DataFrame()
    level_name = level.replace('levels\\/', '')
    level_name= level_name.replace('\\/', '_')
    level_name = level_name.replace('.level', '')
    level_name = level_name.replace(' ', '_')
    print "Processing", level_name
    filename = "Data/LevelFeatures/FullNewLevels/"+level_name+".csv"
    df_time_series = pd.read_table(filename, delimiter = ',') 
    df_time_series['userId'] = df_time_series['userId'].astype('str')
    
    #clean data
    #discrad time taken < 1 minute and time taken > 20 minutes
    #df_level = df_level_full[(df_level_full["current_level"]==level) & ((df_level_full['time_taken_first_attempt'] >=TIME_BIN) | (df_level_full['quit_first_attempt']==0)) & (df_level_full['total_time_taken'] <= MAX_TIME)]
    #valid_users = df_level['userId'].unique()
    #print "number of students", len(valid_users)
    #df_time_series = df_time_series[df_time_series['userId'].isin(valid_users)]
    print "number of log instances", len(df_time_series)
    
    df_agg = pd.DataFrame()
    df_temp = pd.DataFrame(columns=df_time_series.columns)
    count = 0
    record_num = 1
    stud = None

    for idx, row in df_time_series.iterrows():
        if row['userId'] != stud:
            if count > 0: #if not the first student
                #update the last student 
                df_agg = update_df_agg(df_agg, df_temp, count) #todo: calculate aggregates of all

            df_temp = pd.DataFrame(columns=df_time_series.columns)
            df_temp = df_temp.append(row)
            count += 1
            record_num = 1
            stud = row['userId']
            start_time = row["timestamp"]

        if row["timestamp"]-start_time <= TIME_BIN*1000:
            df_temp = df_temp.append(row)

        if row["timestamp"]-start_time > TIME_BIN*1000: #aggregating every 1 minute
            df_agg = update_df_agg(df_agg, df_temp, count)
            df_temp = pd.DataFrame(columns=df_time_series.columns) #flush out for new aggregates
            df_temp = df_temp.append(row)
            count += 1
            record_num += 1
            start_time = row["timestamp"]

    #Student level cross validation - assign fold labels
    #split students - random sampling - train and test
    unique_studs = df_agg["userId"].unique()
    print "Total students", len(unique_studs)
    rand_seq_studs = np.random.choice(unique_studs, len(unique_studs), replace=False)
    fold_dict = {}
    fold_dict[1] = rand_seq_studs[0:int(len(unique_studs)*0.2)]
    fold_dict[2] = rand_seq_studs[int(len(unique_studs)*0.2):int(len(unique_studs)*0.4)]
    fold_dict[3] = rand_seq_studs[int(len(unique_studs)*0.4):int(len(unique_studs)*0.6)]
    fold_dict[4] = rand_seq_studs[int(len(unique_studs)*0.6):int(len(unique_studs)*0.8)]
    fold_dict[5] = rand_seq_studs[int(len(unique_studs)*0.8):]
    print "Number of students in each fold", len(fold_dict[1]), len(fold_dict[2]), len(fold_dict[3]), len(fold_dict[4]), len(fold_dict[5])

    user_dict = {}
    #inverse dict
    for k, v in fold_dict.iteritems():
        for user in v:
            user_dict[user] = k

    for idx, row in df_agg.iterrows():
        df_agg.set_value(idx, "fold", user_dict[row['userId']])

    print "fold sizes (number of data samples)= ", len(df_agg[df_agg["fold"]==1]), len(df_agg[df_agg["fold"]==2]), len(df_agg[df_agg["fold"]==3]), len(df_agg[df_agg["fold"]==4]), len(df_agg[df_agg["fold"]==5])

    for i in range(5):
        fold = df_agg[df_agg["fold"]==i+1]
        print "% quit in fold", i, float(np.sum(fold["quit"]))/len(fold)

    filename = "Data/LevelFeatures/TestFullNewLevels/"+level_name+"_"+str(TIME_BIN)+"s.csv"
    df_agg.to_csv(filename, header=True, index=False) 

Processing p2_downhill
number of log instances 9302
Total students 124
Number of students in each fold 24 25 25 25 25
fold sizes (number of data samples)=  36 50 34 40 49
% quit in fold 0 0.0
% quit in fold 1 0.1
% quit in fold 2 0.0294117647059
% quit in fold 3 0.075
% quit in fold 4 0.163265306122
Processing p2_lead_the_ball
number of log instances 5760
Total students 123
Number of students in each fold 24 25 24 25 25
fold sizes (number of data samples)=  37 30 31 40 47
% quit in fold 0 0.0
% quit in fold 1 0.0666666666667
% quit in fold 2 0.0322580645161
% quit in fold 3 0.1
% quit in fold 4 0.0212765957447
Processing p3_jelly_beans
number of log instances 37473
Total students 122
Number of students in each fold 24 24 25 24 25
fold sizes (number of data samples)=  68 89 116 71 98
% quit in fold 0 0.132352941176
% quit in fold 1 0.123595505618
% quit in fold 2 0.206896551724
% quit in fold 3 0.140845070423
% quit in fold 4 0.397959183673
Processing p3_move_the_rocks
number of log ins

In [ ]:
#COMBINE ALL LEVELS
import os
import pandas as pd
import re

df_ml = pd.DataFrame()
fn = "Data/LevelFeatures/Test"#Aggregated"

for file in os.listdir(fn):
    if file.endswith(".csv"):
        df_temp = pd.read_csv(os.path.join(fn, file))
        df_ml = pd.concat([df_ml, df_temp]) #df_ml2
        
print "Total data samples", len(df_ml)
df_ml.to_csv("Data/LevelFeatures/ModelA_full_level_agnostic_test.csv", header=True, index=False)

In [ ]:
#LEVEL RELATED FEATURES - both the ones that need past data and the ones that don't
import os
import pandas as pd
import re

df_level_feat = pd.read_csv('Data/LevelFeaturesNoQuit.csv')
df_ml = pd.DataFrame()
fn = "Data/LevelFeatures/TestFullNewLevels"#Aggregated"

for file in os.listdir(fn):
    if file.endswith(".csv"):
        df_temp = pd.read_csv(os.path.join(fn, file))
        name = file.replace("_60s.csv","")
        name = name[3:].replace('_',' ')
        df_feat_temp = df_level_feat[df_level_feat["current_level"].str.contains(name)]
        for idx, row in df_temp.iterrows():
            for col in df_feat_temp.columns:
                if col != 'current_level':
                    df_temp.set_value(idx, col, df_feat_temp[col].iloc[0])
        df_ml = pd.concat([df_ml, df_temp]) #df_ml2
        
print "Total data samples", len(df_ml)
df_ml.to_csv("Data/LevelFeatures/ModelC_full_with_level_specific_test_new_all.csv", header=True, index=False)

In [3]:
def update_df(idx, df, df_stud_levels):
    if len(df_stud_levels) == 0:
        df.set_value(idx,'mean_level_restarts_overall_stud', 0)
        df.set_value(idx,'std_level_restarts_overall_stud', 0)
        df.set_value(idx,'mean_ball_resets_overall_stud', 0)
        df.set_value(idx,'std_ball_resets_overall_stud', 0)
        df.set_value(idx,'mean_pause_duration_overall_stud', 0)
        df.set_value(idx,'std_pause_duration_overall_stud', 0)
        df.set_value(idx,'mean_total_freeform_overall_stud', 0)
        df.set_value(idx,'std_total_freeform_overall_stud', 0)
        df.set_value(idx,'mean_total_pendulum_stud', 0)
        df.set_value(idx,'std_total_pendulum_stud', 0)
        df.set_value(idx,'mean_total_lever_overall_stud', 0)
        df.set_value(idx,'std_total_lever_overall_stud', 0)
        df.set_value(idx,'mean_total_ramp_overall_stud', 0)
        df.set_value(idx,'std_total_ramp_overall_stud', 0)
        df.set_value(idx,'mean_total_springboard_overall_stud', 0)
        df.set_value(idx,'std_total_springboard_overall_stud', 0)
        df.set_value(idx,'mean_number_pauses_overall_stud', 0)
        df.set_value(idx,'std_number_pauses_overall_stud', 0)
        df.set_value(idx,'mean_attempt_stud', 0)
        df.set_value(idx,'std_attempt_stud', 0)
        df.set_value(idx,'mean_time_spent_stud', 0)
        df.set_value(idx,'std_time_spent_stud_stud', 0)
        df.set_value(idx,'mean_badges_overall_stud', 0)
        df.set_value(idx,'std_badges_overall_stud', 0)
        df.set_value(idx,'num_R_levels_stud', 0)
        df.set_value(idx,'num_P_levels_stud', 0)
        df.set_value(idx,'num_L_levels_stud', 0)
        df.set_value(idx,'num_SB_levels_stud', 0)
        df.set_value(idx,'num_EcT_levels_stud', 0)
        df.set_value(idx,'num_PoT_levels_stud', 0)
        df.set_value(idx,'num_NL_levels_stud', 0)
        df.set_value(idx,'num_levels', 0)
        df.set_value(idx,'num_levels_quit_atleast_once', 0)
        df.set_value(idx,'num_R_levels_stud_solved', 0)
        df.set_value(idx,'num_L_levels_stud_solved', 0)
        df.set_value(idx,'num_SB_levels_stud_solved', 0)
        df.set_value(idx,'num_P_levels_stud_solved', 0)
        df.set_value(idx,'num_EcT_levels_stud_solved', 0)
        df.set_value(idx,'num_PoT_levels_stud_solved', 0)
        df.set_value(idx,'num_NL_levels_stud_solved', 0)
        return
    
    df.set_value(idx,'mean_level_restarts_overall_stud', np.mean(df_stud_levels['level_restarts_overall']))
    df.set_value(idx,'std_level_restarts_overall_stud', np.std(df_stud_levels['level_restarts_overall']))
    df.set_value(idx,'mean_ball_resets_overall_stud', np.mean(df_stud_levels['ball_resets_overall']))
    df.set_value(idx,'std_ball_resets_overall_stud', np.std(df_stud_levels['ball_resets_overall']))
    df.set_value(idx,'mean_pause_duration_overall_stud', np.mean(df_stud_levels['total_pause_duration_overall']))
    df.set_value(idx,'std_pause_duration_overall_stud', np.std(df_stud_levels['total_pause_duration_overall']))
    df.set_value(idx,'mean_total_freeform_overall_stud', np.mean(df_stud_levels['total_freeform_overall']))
    df.set_value(idx,'std_total_freeform_overall_stud', np.std(df_stud_levels['total_freeform_overall']))
    df.set_value(idx,'mean_total_pendulum_stud', np.mean(df_stud_levels['total_pendulum_overall']))
    df.set_value(idx,'std_total_pendulum_stud', np.std(df_stud_levels['total_pendulum_overall']))
    df.set_value(idx,'mean_total_lever_overall_stud', np.mean(df_stud_levels['total_lever_overall']))
    df.set_value(idx,'std_total_lever_overall_stud', np.std(df_stud_levels['total_lever_overall']))
    df.set_value(idx,'mean_total_ramp_overall_stud', np.mean(df_stud_levels['total_ramp_overall']))
    df.set_value(idx,'std_total_ramp_overall_stud', np.std(df_stud_levels['total_ramp_overall']))
    df.set_value(idx,'mean_total_springboard_overall_stud', np.mean(df_stud_levels['total_springboard_overall']))
    df.set_value(idx,'std_total_springboard_overall_stud', np.std(df_stud_levels['total_springboard_overall']))
    df.set_value(idx,'mean_number_pauses_overall_stud', np.mean(df_stud_levels['number_pauses_overall']))
    df.set_value(idx,'std_number_pauses_overall_stud', np.std(df_stud_levels['number_pauses_overall']))
    df.set_value(idx,'mean_attempt_stud', np.mean(df_stud_levels['attempt']))
    df.set_value(idx,'std_attempt_stud', np.std(df_stud_levels['attempt']))
    df.set_value(idx,'mean_time_spent_stud', np.mean(df_stud_levels['record_num'])) #proxy for time taken
    df.set_value(idx,'std_time_spent_stud_stud', np.std(df_stud_levels['record_num']))
    df.set_value(idx,'mean_badges_overall_stud', np.mean(df_stud_levels['badges_overall']))
    df.set_value(idx,'std_badges_overall_stud', np.std(df_stud_levels['badges_overall']))
    df.set_value(idx,'num_R_levels_stud', len(df_stud_levels[df_stud_levels['R']==1]))
    df.set_value(idx,'num_P_levels_stud', len(df_stud_levels[df_stud_levels['P']==1]))
    df.set_value(idx,'num_L_levels_stud', len(df_stud_levels[df_stud_levels['L']==1]))
    df.set_value(idx,'num_SB_levels_stud', len(df_stud_levels[df_stud_levels['SB']==1]))
    df.set_value(idx,'num_EcT_levels_stud', len(df_stud_levels[df_stud_levels['EcT']==1]))
    df.set_value(idx,'num_PoT_levels_stud', len(df_stud_levels[df_stud_levels['PoT']==1]))
    df.set_value(idx,'num_NL_levels_stud', len(df_stud_levels[df_stud_levels['NL']==1]))
    df.set_value(idx,'num_levels', len(df_stud_levels))
    df.set_value(idx,'num_levels_quit_atleast_once', len(df_stud_levels[df_stud_levels['quits_num'] > 0]))
    df.set_value(idx,'num_R_levels_stud_solved', len(df_stud_levels[(df_stud_levels['R']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_L_levels_stud_solved', len(df_stud_levels[(df_stud_levels['L']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_SB_levels_stud_solved', len(df_stud_levels[(df_stud_levels['SB']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_P_levels_stud_solved', len(df_stud_levels[(df_stud_levels['P']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_EcT_levels_stud_solved', len(df_stud_levels[(df_stud_levels['EcT']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_PoT_levels_stud_solved', len(df_stud_levels[(df_stud_levels['PoT']==1)&(df_stud_levels['quit']==0)]))
    df.set_value(idx,'num_NL_levels_stud_solved', len(df_stud_levels[(df_stud_levels['NL']==1)&(df_stud_levels['quit']==0)]))   